In [1]:
from DataReader import DataReaderChallangePolimi2017
import implicit
from CB_asy import CB_asymmetric_cosine
from CF_IB_asy import CF_IB_asymmetric_cosine
from CF_UB_asy import CF_UB_asymmetric_cosine
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,TFIDFRecommender)
from sklearn.preprocessing import normalize
import numpy as np
import scipy.sparse as sp
import pickle

evaluation = True
rebuild = False
dr = DataReaderChallangePolimi2017(evaluation=evaluation,rebuild=rebuild)

urm = dr.getURM_csr()
t_p = dr.target_playlists
t_t = dr.target_tracks
if evaluation:
    filesslim = "./DumpData/sim_bpr_10e_100k2.npz" # or "./DumpData/sim_bpr_10e_100k.npz"
else:
    filesslim = "./DumpData/sim_bpr_10e_100k_fulltrain.npz"
    

Date Reader Challange Polimi 2017
Loading data from ./DumpData/DataReaderTrainTestDump
Dataset train-test loaded


In [68]:
#track-album
icm = dr.getICM_csr(albums=True,artists=False,tags=False)
#icm = normalize(icm,axis=0)
model = BM25Recommender(K=30)
model.fit(icm)
s = model.similarity
s.setdiag(0)
r_al = urm*s.T
r_al = normalize(r_al, axis=0)
if evaluation: print dr.evaluateMAP(r_al,verbose=False)
r_al = dr.reduceRM(r_al)

0.0712471111111


In [70]:
#track-artist
icm = dr.getICM_csr(albums=False,artists=True,tags=False)
icm = normalize(icm,axis=0)
model = BM25Recommender(K=68)
model.fit(icm)
s = model.similarity
s.setdiag(0)
r_ar = urm*s.T
r_ar = normalize(r_ar, axis=0)
if evaluation: print dr.evaluateMAP(r_ar,verbose=False)
r_ar = dr.reduceRM(r_ar)

0.0710406388889


In [4]:
#playtlist-album
icm_al = dr.getUCM_csr(albums=True,artists=False)
model = BM25Recommender(K=120)
model.fit(icm_al)
s = model.similarity
r_pal = s.T*urm
r_pal = normalize(r_pal, axis=0)
if evaluation: print dr.evaluateMAP(r_pal,verbose=False)
r_pal = dr.reduceRM(r_pal)

0.0724248333333


In [5]:
#playlist-artist
icm_ar = dr.getUCM_csr(albums=False,artists=True)
model = BM25Recommender(K=100)
model.fit(icm_ar)
s = model.similarity
r_par = s.T*urm
r_par = normalize(r_par, axis=0)
if evaluation: print dr.evaluateMAP(r_par,verbose=False)
r_par = dr.reduceRM(r_par)

0.0607021666667


In [6]:
#album and artist
icm = dr.getICM_csr(albums=True,artists=True,tags=False)
icm = normalize(icm,axis=0)
model = BM25Recommender(K=58)
model.fit(icm)
s = model.similarity
r_al_ar = urm*s.T #or s
r_al_ar = normalize(r_al_ar, axis=0)
if evaluation: print dr.evaluateMAP(r_al_ar,verbose=False)
r_al_ar = dr.reduceRM(r_al_ar)

0.0790871666667


In [7]:
#tags
icm = dr.getICM_csr(albums=False,artists=False,tags=True)
icm = normalize(icm,axis=1)   # little improvment
model = BM25Recommender(K = 15)
model.fit(icm)
s = model.similarity
r_tag = urm*s
r_tag = normalize(r_tag, axis=0)
if evaluation: print dr.evaluateMAP(r_tag, verbose = False)
r_tag = dr.reduceRM(r_tag)

0.0377463611111


In [49]:
#cf ub
model = BM25Recommender(K=60)
model.fit(urm)
s = model.similarity
r_cfub = s.T * urm
r_cfub = normalize(r_cfub, axis=1)
if evaluation: print dr.evaluateMAP(r_cfub, verbose = False)
r_cfub = dr.reduceRM(r_cfub)

0.0661010277778


In [50]:
#cf ib
model = BM25Recommender(K=100)
model.fit(urm.T)
s = model.similarity
#s.setdiag(0)
r_cfib = urm * s.T
r_cfib = normalize(r_cfib, axis=0)
if evaluation: print dr.evaluateMAP(r_cfib, verbose = False)
r_cfib = dr.reduceRM(r_cfib)

0.0729845


In [482]:
#r_bpr = dr.reduceRM(urm * dr.load_sparse_csr(filesslim))
#if evaluation: print dr.evaluateMAP(r_bpr, verbose=False)

In [11]:
#############start with the mix#################

In [12]:
######### cb

In [13]:
r_cb1 = r_al_ar + 0.25*r_tag 
if evaluation: print dr.evaluateMAP(r_cb1, verbose = False)

0.0811120277778


In [14]:
#not used atm
#r_totcb = normalize(r_cb1,axis=1) + 0.2* r_al + 0.15 * r_ar
#if evaluation: print dr.evaluateMAP(r_totcb, verbose = False)

In [15]:
########## cf

In [459]:
#total cf
r_totcf = 1.8*r_cfib + r_cfub
if evaluation: print dr.evaluateMAP(r_totcf, verbose = False)

0.0781461944444


In [460]:
########## all

In [461]:
r_tot = r_cb1 + 1.80 * r_totcf
if evaluation: print dr.evaluateMAP(r_tot,verbose = False)

0.102650472222


In [19]:
#not used atm
#r_tot2 = r_tot + 5* r_bpr

#if evaluation: print dr.evaluateMAP(r_tot2,verbose = False)

In [20]:
######improve total a little

In [21]:
r_tot2 = r_tot + 0.25*r_pal + 0.25*r_par
if evaluation: print dr.evaluateMAP(r_tot2,verbose = False)

0.104498


In [22]:
r_tot3 = r_tot2 + 0.10*r_al + 0.10*r_ar
if evaluation: print dr.evaluateMAP(r_tot3,verbose = False)

0.104830638889


In [ ]:
###################################

In [19]:
#owners
ucm = dr.getOwnersUCM_csr()
ucm = normalize(ucm,axis=0)
model = BM25Recommender(K=105)#110
model.fit(ucm)
s = model.similarity
r_o = s.T*urm
r_o = normalize(r_o, axis=0)
if evaluation: print dr.evaluateMAP(r_o,verbose=False)
r_o = dr.reduceRM(r_o)

0.0225449166667


In [24]:
icm_o = dr.getOwnersICM_csr(ones=True)

In [ ]:
################################ 0.24 tag 1.25 cfs 1.82 sum

In [349]:
r1 = (1.250*r_cfib+r_cfub)

In [350]:
r2 = r_al_ar + 0.240 * r_tag

In [351]:
r_tot = 1.820*r1 + r2

In [352]:
r_tot3 = r_tot + 0.150*r_al + 0.120*r_ar
if evaluation: print dr.evaluateMAP(r_tot3,verbose = False)

0.104666888889


In [353]:
r_tot4 = r_tot3 + 0.750 * r_o
if evaluation: print dr.evaluateMAP(r_tot4,verbose = False)

0.109852138889


In [ ]:
###############################

In [71]:
r_tot4 = 1.809 * 1.250 * r_cfib + 1.820 * r_cfub + r_al_ar + 0.240 * r_tag + 0.751 * r_o + 0.150*r_al + 0.119*r_ar
if evaluation: print dr.evaluateMAP(r_tot4,verbose = False)

0.109926027778


In [ ]:
#################################

In [ ]:
[2.26125,1.820,1.0,0.240,0.751,0.0150,0.119]
[2.74711712e-04,   1.83011415e-04,   2.13292590e-05,
         1.25368191e-05,   8.23875897e-05,   5.61734242e-05,
         7.94264166e-05]

In [64]:
from scipy.optimize import minimize

In [57]:
def fun(p):
    r_tot = p[0] * r_cfib + p[1] * r_cfub + p[2] * r_al_ar + p[3]* r_tag + p[4] * r_o + p[5]*r_al + p[6]*r_ar
    score = dr.evaluateMAP(r_tot,verbose = False)
    print score
    return -score

def fun2(p):
    r_tot = p[0] * r_al_ar + p[1]* r_tag  + p[2]*r_al + p[3]*r_ar
    score = dr.evaluateMAP(r_tot,verbose = False)
    print score
    return -score
#[  1.69027787e-04,   4.62730153e-05,   2.41163797e-04,   3.63955333e-04]

def fun3(p):
    r_tot = p[0] * r_cfib + p[1] * r_cfub 
    score = dr.evaluateMAP(r_tot,verbose = False)
    print score
    return -score
#[  3.60369489e-04,   2.51829624e-08]

def fun4(p):
    r_tot = p[0] * r_1 + p[1] * r_2 + p[2] * r_o
    score = dr.evaluateMAP(r_tot,verbose = False)
    print score
    return -score

In [40]:
p = [  1.69027787e-04,   4.62730153e-05,   2.41163797e-04,   3.63955333e-04]
r_1 = p[0] * r_al_ar + p[1]* r_tag  + p[2]*r_al + p[3]*r_ar
print dr.evaluateMAP(r_1,verbose = False)

p = [3.60369489e-04,   2.51829624e-08]
r_2 = p[0] * r_cfib + p[1] * r_cfub
print dr.evaluateMAP(r_2,verbose = False)

0.0846163611111
0.0786673888889


In [ ]:
#################################

In [ ]:
for p in np.arange(0.70,0.90,0.01):
    r_tot5 = r_tot4 + p * r_o
    score = dr.evaluateMAP(r_tot5,verbose = False)
    print ("k = {:1.2f} \t\tmap = {:1.5f}").format(p,score)

In [77]:
rec = dr.getAllRecommendations(r_tot)
dr.buildSubmissionFile(recommendation=rec, file_name="ci_si_prova_3_mix_0-0-0-0") ####<------name

User 0 of 10000
User 2500 of 10000
User 5000 of 10000
User 7500 of 10000
Recommendations done
Creating submission file in: ./Submissions/ci_si_prova_3_mix_0-0-0-0
Done


In [ ]:
## actually not used

In [47]:
#collaborative filtering item based asym
m_cf_ib = CF_IB_asymmetric_cosine(urm, k=200, alpha=0.30, q=1, m=0)
s = m_cf_ib.s
r_cfib_asy = urm*s
if evaluation: print dr.evaluateMAP(r_cfib_asy, verbose=False)
# reduce matrix
r_cfib_asy_more = normalize(r_cfib,axis=1) #4.75
r_cfib_asy_less = normalize(r_cfib,axis=0)#2.75 #######axis 0 better in sum

r_cfib_asy = dr.reduceRM(r_cfib_asy)
r_cfib_asy_more = dr.reduceRM(r_cfib_asy_more)
r_cfib_asy_less = dr.reduceRM(r_cfib_asy_less)

Start asymmetric cosine item-based model..
Keep only k-similar item..
Building sparse matrix..
Start building rating matrix..
Normalizing rating matrix..
Model asymmetric cosine item-based done
0.0720835555556
